<a href="https://colab.research.google.com/github/savindumahasen/AI-Detection-System-/blob/main/Notebook.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [83]:
# install the necessary libraries
!pip install pandas
!pip install numpy

In [84]:
# import necessary libraries
import pandas as pd
import numpy as np
import csv

In [85]:
dataset = pd.read_excel("/content/Dataset.xlsx")

dataset.head()

,sr.no,text,source,label_name,label_id
0,0,"Produced by Chris Young Ritzen, Tamara Weitzma...",openwebtext,human-written,1
1,1,"Lawrence Eugene Doby (December 13, 1923 – June...",openwebtext,human-written,1
2,2,Shock and Awe\n\nGabe Newell is about to make ...,openwebtext,human-written,1
3,3,The Best Answers to Tough Interview Questions\...,openwebtext,human-written,1
4,4,Contents:\n\n1.) Preamble\n\n2.) Understand Yo...,openwebtext,human-written,1


In [86]:
## get the information  of dataset
dataset.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 11580 entries, 0 to 11579
Data columns (total 5 columns):
 #   Column      Non-Null Count  Dtype 
---  ------      --------------  ----- 
 0   sr.no       11580 non-null  int64 
 1   text        11580 non-null  object
 2   source      11580 non-null  object
 3   label_name  11580 non-null  object
 4   label_id    11580 non-null  int64 
dtypes: int64(2), object(3)
memory usage: 452.5+ KB


In [87]:
## length of the dataset
len(dataset)

11580

In [88]:
# get the value counts of label_id
dataset['label_id'].value_counts()

,count
label_id,
1,5790
0,5790


In [89]:
## split the dataset into  train and test set
from sklearn.model_selection  import train_test_split
train,test=train_test_split(dataset,test_size=0.1, random_state=42, shuffle=False)
train.shape,test.shape

((10422, 5), (1158, 5))

In [90]:
train.head()

,sr.no,text,source,label_name,label_id
0,0,"Produced by Chris Young Ritzen, Tamara Weitzma...",openwebtext,human-written,1
1,1,"Lawrence Eugene Doby (December 13, 1923 – June...",openwebtext,human-written,1
2,2,Shock and Awe\n\nGabe Newell is about to make ...,openwebtext,human-written,1
3,3,The Best Answers to Tough Interview Questions\...,openwebtext,human-written,1
4,4,Contents:\n\n1.) Preamble\n\n2.) Understand Yo...,openwebtext,human-written,1


In [91]:
test.head()

,sr.no,text,source,label_name,label_id
10422,10422,I have heard a lot about the abuse thing about...,paraphrase,ai-generated,0
10423,10423,It really pisses me off when humans are calle...,paraphrase,ai-generated,0
10424,10424,"Recently, I often see the uproar of people str...",paraphrase,ai-generated,0
10425,10425,"Before the negative voices are showered at me,...",paraphrase,ai-generated,0
10426,10426,If I don't talk about it in public or with my ...,paraphrase,ai-generated,0


## Data Preprocessing

In [92]:
train.shape

(10422, 5)

In [93]:
## check  the duplicate values of train set
train.duplicated().sum()

np.int64(0)

In [94]:
## check the null value of train set
train.isnull().sum()

,0
sr.no,0
text,0
source,0
label_name,0
label_id,0


In [95]:
train['label_id'].head()

,label_id
0,1
1,1
2,1
3,1
4,1


In [96]:
## seperate the target variable from indepdentdant variables
train_set=train.drop(['label_id','label_name','source','sr.no'],axis=True)
train_set.head()

,text
0,"Produced by Chris Young Ritzen, Tamara Weitzma..."
1,"Lawrence Eugene Doby (December 13, 1923 – June..."
2,Shock and Awe\n\nGabe Newell is about to make ...
3,The Best Answers to Tough Interview Questions\...
4,Contents:\n\n1.) Preamble\n\n2.) Understand Yo...


### Text preprocessing

In [97]:
## import text preprocessing libraries
import re
import string

In [98]:
## convert uppercase letters into  lowercase letters

In [99]:
train['text']=train['text'].apply(lambda x: " ".join(x.lower() for x in x.split()))

In [100]:
train['text'].head()

,text
0,"produced by chris young ritzen, tamara weitzma..."
1,"lawrence eugene doby (december 13, 1923 – june..."
2,shock and awe gabe newell is about to make a p...
3,the best answers to tough interview questions ...
4,contents: 1.) preamble 2.) understand your soc...


In [101]:
## remove the links from text

In [102]:
train['text']=train['text'].apply(lambda x: " ".join(re.sub(r'^https?:\/\/.*[\r\n]*','', x,flags=re.MULTILINE) for x in x.split()))
train['text'].head()

,text
0,"produced by chris young ritzen, tamara weitzma..."
1,"lawrence eugene doby (december 13, 1923 – june..."
2,shock and awe gabe newell is about to make a p...
3,the best answers to tough interview questions ...
4,contents: 1.) preamble 2.) understand your soc...


In [103]:
## remove punctuations

In [104]:
string.punctuation

'!"#$%&\'()*+,-./:;<=>?@[\\]^_`{|}~'

In [105]:
def remove_punctuation(text):
  for punctuations in string.punctuation:
     text=text.replace(punctuations,'')
  return text

In [106]:
train['text']= train['text'].apply(remove_punctuation)
train['text'].head()

,text
0,produced by chris young ritzen tamara weitzman...
1,lawrence eugene doby december 13 1923 – june 1...
2,shock and awe gabe newell is about to make a p...
3,the best answers to tough interview questions ...
4,contents 1 preamble 2 understand your social s...


In [107]:
## remove the numbers from text

In [108]:
train["text"]=train["text"].str.replace('/d+','', regex=True)

In [109]:
train['text'].head()

,text
0,produced by chris young ritzen tamara weitzman...
1,lawrence eugene doby december 13 1923 – june 1...
2,shock and awe gabe newell is about to make a p...
3,the best answers to tough interview questions ...
4,contents 1 preamble 2 understand your social s...


In [110]:
## remove  stopwords

In [111]:
import nltk

In [112]:
nltk.download('stopwords', download_dir="words/")

[nltk_data] Downloading package stopwords to words/...
[nltk_data]   Package stopwords is already up-to-date!


True

In [113]:
with open ('words/corpora/stopwords/english', 'r') as file:
    sw=file.read().splitlines()

In [114]:
sw

['a',
 'about',
 'above',
 'after',
 'again',
 'against',
 'ain',
 'all',
 'am',
 'an',
 'and',
 'any',
 'are',
 'aren',
 "aren't",
 'as',
 'at',
 'be',
 'because',
 'been',
 'before',
 'being',
 'below',
 'between',
 'both',
 'but',
 'by',
 'can',
 'couldn',
 "couldn't",
 'd',
 'did',
 'didn',
 "didn't",
 'do',
 'does',
 'doesn',
 "doesn't",
 'doing',
 'don',
 "don't",
 'down',
 'during',
 'each',
 'few',
 'for',
 'from',
 'further',
 'had',
 'hadn',
 "hadn't",
 'has',
 'hasn',
 "hasn't",
 'have',
 'haven',
 "haven't",
 'having',
 'he',
 "he'd",
 "he'll",
 'her',
 'here',
 'hers',
 'herself',
 "he's",
 'him',
 'himself',
 'his',
 'how',
 'i',
 "i'd",
 'if',
 "i'll",
 "i'm",
 'in',
 'into',
 'is',
 'isn',
 "isn't",
 'it',
 "it'd",
 "it'll",
 "it's",
 'its',
 'itself',
 "i've",
 'just',
 'll',
 'm',
 'ma',
 'me',
 'mightn',
 "mightn't",
 'more',
 'most',
 'mustn',
 "mustn't",
 'my',
 'myself',
 'needn',
 "needn't",
 'no',
 'nor',
 'not',
 'now',
 'o',
 'of',
 'off',
 'on',
 'once',
 'on

In [115]:
train["text"]= train["text"].apply(lambda x: " ".join(x for x in x.split() if x not in sw))

In [116]:
train['text'].head()

,text
0,produced chris young ritzen tamara weitzman pa...
1,lawrence eugene doby december 13 1923 – june 1...
2,shock awe gabe newell make promise 11am overca...
3,best answers tough interview questions tell re...
4,contents 1 preamble 2 understand social surrou...


In [117]:
## remove the inflectional form from text

In [118]:
from nltk.stem import PorterStemmer
ps=PorterStemmer()

In [119]:
train["text"]=train["text"].apply(lambda x: " ".join(ps.stem(x) for x in x.split()))

In [120]:
train['text'].head()

,text
0,produc chri young ritzen tamara weitzman patti...
1,lawrenc eugen dobi decemb 13 1923 – june 18 20...
2,shock awe gabe newel make promis 11am overcast...
3,best answer tough interview question tell real...
4,content 1 preambl 2 understand social surround...


In [121]:
train['text'].tail()

,text
10417,im big star war fan cinema eu purpos discuss w...
10418,edit ive provid delta thread answer provid del...
10419,believ abbott govern polici attempt polici who...
10420,inspir articl outrag someon build someth home ...
10421,littl background 21 year old femal devot first...


In [122]:
from nltk.stem import WordNetLemmatizer
from nltk.corpus import wordnet
lemmertizer=WordNetLemmatizer()

# Download required data once
nltk.download('wordnet')
nltk.download('omw-1.4')

train["text"]=train["text"].apply(lambda x: " ".join(lemmertizer.lemmatize(x, pos='v') for x in x.split()))
train['text'].tail()


[nltk_data] Downloading package wordnet to /root/nltk_data...
[nltk_data]   Package wordnet is already up-to-date!
[nltk_data] Downloading package omw-1.4 to /root/nltk_data...
[nltk_data]   Package omw-1.4 is already up-to-date!


,text
10417,im big star war fan cinema eu purpos discuss w...
10418,edit ive provid delta thread answer provid del...
10419,believ abbott govern polici attempt polici who...
10420,inspir articl outrag someon build someth home ...
10421,littl background 21 year old femal devot first...


### Building the vocabulary

In [123]:
from collections import Counter
vocab=Counter()

In [124]:
for sentence in train['text']:
   vocab.update(sentence.split())

In [125]:
len(vocab)

134975

In [126]:
vocab

Counter({'produc': 2343,
         'chri': 393,
         'young': 1847,
         'ritzen': 1,
         'tamara': 1,
         'weitzman': 1,
         'patti': 33,
         'aronofski': 2,
         'nov': 76,
         '20': 1520,
         '2014': 822,
         'juri': 287,
         'seminol': 3,
         'counti': 641,
         'florida': 386,
         'take': 13439,
         'less': 4218,
         'three': 3191,
         'hour': 2253,
         'convict': 467,
         'anita': 174,
         'smithey': 65,
         'seconddegre': 13,
         'murder': 945,
         'admit': 727,
         'shoot': 1990,
         'kill': 2629,
         'estrang': 13,
         'husband': 440,
         'robert': 545,
         'cline': 31,
         'say': 22257,
         'rape': 916,
         'think': 15036,
         'go': 15012,
         'tell': 5294,
         'truth': 740,
         'terribl': 482,
         'injustic': 127,
         'do': 2433,
         '48': 176,
         'lay': 477,
         'fact': 3980,


In [127]:
def save_vocabulary(lines, filename):
    data = '\n'.join(lines)
    file = open(filename, 'w', encoding="utf-8")
    file.write(data)
    file.close()

save_vocabulary(vocab, 'vocabulary/vocabulary.txt')

In [128]:
## sperate the target vairable from another independent variables

In [129]:
x=train['text'];
y=train['label_id']

## Split the train set into  train and test set

In [132]:
from sklearn.model_selection import train_test_split
X_train,X_test,y_train,y_test=train_test_split(x,y,test_size=0.1, random_state=42,shuffle=False)
X_train.shape, X_test.shape, y_train.shape,y_test.shape

((9379,), (1043,), (9379,), (1043,))

In [133]:
import numpy as np

def vectorizer(ds, vocabulary):
    vectorizer_list = []
    vocab_set = set(vocabulary)  # Use a set for faster lookup

    for sentence in ds:
        sentence_list = np.zeros(len(vocabulary), dtype=np.float16)  # Use np.float16 to reduce memory usage
        sentence_words = set(sentence.split())  # Split and convert sentence to a set to speed up membership checking

        for i, word in enumerate(vocabulary):
            if word in sentence_words:  # Check if word exists in the sentence
                sentence_list[i] = 1

        vectorizer_list.append(sentence_list)

    # Convert the list to a numpy array only once
    vectorizer_list_new = np.asarray(vectorizer_list, dtype=np.float16)

    return vectorizer_list_new

In [135]:
vectorized_x_train = vectorizer(X_train,vocab)

In [136]:
vectorized_x_train

array([[1., 1., 1., ..., 0., 0., 0.],
       [0., 0., 1., ..., 0., 0., 0.],
       [1., 0., 1., ..., 0., 0., 0.],
       ...,
       [0., 0., 1., ..., 0., 0., 0.],
       [1., 0., 0., ..., 0., 0., 0.],
       [0., 0., 0., ..., 0., 0., 0.]], dtype=float16)

In [138]:
vectorized_x_test=vectorizer(X_test,vocab)

In [139]:
vectorized_x_test

array([[0., 0., 0., ..., 0., 0., 0.],
       [0., 0., 1., ..., 0., 0., 0.],
       [0., 0., 0., ..., 0., 0., 0.],
       ...,
       [0., 0., 0., ..., 0., 0., 0.],
       [0., 0., 0., ..., 0., 0., 0.],
       [0., 0., 0., ..., 0., 0., 0.]], dtype=float16)

In [142]:
vectorized_x_train.shape

(9379, 134975)

In [141]:
y_train.shape

(9379,)

In [143]:
vectorized_x_test.shape

(1043, 134975)

In [144]:
y_test.shape

(1043,)